In [12]:
import json
from geopandas import GeoDataFrame
from shapely.geometry import Polygon, Point, LineString
import geopandas as gpd
import pandas as pd
import re

import folium
from folium.plugins import MarkerCluster

### Folium Map

Читаю датасет с координатами событий и конвертирую его в json, предварительно объявив его GeoDataFram'ом. С ним намного удобнее работать :)

In [9]:
event_coords = pd.read_csv('event_coo.csv')
geometry = [Point(xy) for xy in zip(event_coords.place_lon, event_coords.place_lat)]
event_coords = GeoDataFrame(event_coords, geometry=geometry)
json_coords = json.loads(event_coords.to_json())['features']

Читаю нужные колонки из сета по событиям

In [4]:
event = pd.read_csv('D:/event.csv', usecols = ['city', 'begin', 'id', 'name', 'place', 'status',])
event.head(2)

,begin,city,id,name,place,status
0,2013-04-03 12:00:00,Каменец-Подольский,61435,"Деловая игра ""Железный предприниматель"" Камене...",Место и адрес проведения уточняются.,ok
1,2013-04-06 15:00:00,Москва,61436,"Лекция Елизаветы Муратовой «Фотокомпозиция, цв...","ул. Большая Полянка, 65/74, стр. 1. FAQcafe",ok


Выбираем Москву (как пример) и мерджим с координатами событий

In [5]:
event_moscow = event[event.city=='Москва'] # 
event_moscow = pd.merge(event_moscow, event_coords, how='left', left_on='id', right_on='event_id')

Дропнем все значения, где не были записаны координаты, такие точки нас не интересуют

In [7]:
event_moscow = event_moscow.dropna(subset=['geometry'], how='all')
event_moscow.head(2)

,begin,city,id,name,place,status,event_id,place_lat,place_lon,subw_lat,subw_lon,geometry
2,2013-04-03 12:00:00,Москва,61440,Встреча с пайщиками,"ул.Сущевская, д.21, подъезд 2, 3 этаж, офис 2,...",ok,61440.0,55.78231,37.600494,55.781788,37.598736,POINT (37.60049438476563 55.78231048583984)
4,2013-04-08 19:00:00,Москва,61453,Игорь Оболенский. Украсть Пиросмани,"ул. Дружинниковская, 11/2",ok,61453.0,55.75753,37.575027,55.760212,37.577209,POINT (37.57502746582031 55.75753021240234)


У json'a созданного выше есть свои id, а внутри блоков запрятан event_id. Создадим словарь event_id -> json_id. Пригодится при рисовании карты

In [10]:
json_ids = []
event_ids = []
for id in range(len(json_coords)):
    json_ids.append(int(json_coords[id]['id']))
    event_ids.append(int(json_coords[id]['properties']['event_id']))
id_map_dict = dict(zip(event_ids,json_ids))

Создаем карту с центром в центра Москвы, выбираем 999 событий и делаем + строим их кластеризацию. 
Всплывающее окно представляет собой html вставку и по-хорошему надо все значки типа кавычек и др. заключать в соответствующие теги. Для просто ты я просто убрал все символы кроме букв.

Карта готова. :)

In [13]:
m = folium.Map(location=[55.764414, 37.647859], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)
for event_id in list(event_moscow.id.values)[:1000]:

    name = re.sub('\W+',' ',event_moscow[event_moscow.id==event_id].name.values[0])
    place = re.sub('\W+',' ',event_moscow[event_moscow.id==event_id].place.values[0])
    city = re.sub('\W+',' ',event_moscow[event_moscow.id==event_id].city.values[0])
    date = re.sub('\W+',' ',event_moscow[event_moscow.id==event_id].begin.values[0])
    latitude = event_moscow[event_moscow.id==event_id].place_lon.values[0]
    longitude = event_moscow[event_moscow.id==event_id].place_lat.values[0]
    
    #print(city, place, date, name, latitude, longitude)
    event_description = folium.Html("""
    <b>Город</b>: {} 
    <br><b>Адрес</b>: {} 
    <br><b>Дата</b>: {} 
    <br><br><b>Описание</b>: {} 
    <br><br><b>Широта</b>: {} 
    <br><b>Долгота</b>: {} """.format(city, place, date, name, latitude, longitude), script=True)

    popup = folium.Popup(event_description, max_width=2650)
    json_id = id_map_dict[event_id]
    folium.Marker(list(reversed(json_coords[json_id]['geometry']['coordinates'])), 
                        popup=popup, 
                 #       radius=1, 
                 #       color='#000000'
                 ).add_to(marker_cluster)

m.save('Moscow Events Map Top-1000.html')
m